In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 85% 103M/121M [00:00<00:00, 233MB/s] 
100% 121M/121M [00:00<00:00, 154MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/face-expression-recognition-dataset.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
!pip install Keras-Preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 627.2 kB/s eta 0:00:00


In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [ ]:
TRAIN_DIR = '/content/images/train'
TEST_DIR = '/content/images/validation'

In [ ]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [ ]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

In [ ]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

In [ ]:
def extract_features(images):
    features = []
    for image in images:
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features

In [ ]:
train_features = extract_features(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [ ]:
test_features = extract_features(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [ ]:
x_train = train_features/255.0
x_test = test_features/255.0

In [ ]:
labelEncoder = LabelEncoder()
labelEncoder.fit(train['label'])

LabelEncoder()

In [ ]:
y_train = labelEncoder.transform(train['label'])
y_test = labelEncoder.transform(test['label'])

In [ ]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [ ]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [ ]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 250, validation_data = (x_test,y_test))

Epoch 1/100
226/226 [==============================] - 12s 51ms/step - loss: 0.5359 - accuracy: 0.8151 - val_loss: 1.1293 - val_accuracy: 0.6339
Epoch 2/100
226/226 [==============================] - 11s 50ms/step - loss: 0.5305 - accuracy: 0.8140 - val_loss: 1.1120 - val_accuracy: 0.6344
Epoch 3/100
226/226 [==============================] - 11s 50ms/step - loss: 0.5335 - accuracy: 0.8165 - val_loss: 1.1144 - val_accuracy: 0.6378
Epoch 4/100
226/226 [==============================] - 11s 50ms/step - loss: 0.5364 - accuracy: 0.8125 - val_loss: 1.0840 - val_accuracy: 0.6332
Epoch 5/100
226/226 [==============================] - 11s 51ms/step - loss: 0.5306 - accuracy: 0.8156 - val_loss: 1.1208 - val_accuracy: 0.6371
Epoch 6/100
226/226 [==============================] - 12s 51ms/step - loss: 0.5139 - accuracy: 0.8198 - val_loss: 1.1094 - val_accuracy: 0.6364
Epoch 7/100
226/226 [==============================] - 12s 52ms/step - loss: 0.5222 - accuracy: 0.8193 - val_loss: 1.0997 - val_ac

In [ ]:
model_json = model.to_json()
with open("emotiondetectornew1.json",'w') as json_file:
    json_file.write(model_json)
model.save("/content/drive/MyDrive/emotiondetectornew1.h5")